In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0,'../../modules')

In [10]:
import numpy as np
import factors
from factors_inference import sum_product_variable_elimination

In [83]:
rain_factor = factors.Factor(["it will rain"],[2])
bus_factor = factors.Factor(["bus is early"],[2])
decision = factors.Factor(["try to catch the bus"],[2])
catch_bus = factors.Factor(["caught bus","bus is early","try to catch the bus"],[2,2,2])
arrive_dry = factors.Factor(["dry","it will rain","caught bus"],[2,2,2])
arrive_late = factors.Factor(["late","caught bus","try to catch the bus"],[2,2,2])
rain_factor.set_all([0.7,0.3]) # 30% chance of rain
bus_factor.set_all([0.85,0.15]) # 15% chance bus is early
catch_bus.set_all([1.0,0.1,1.0,0.6,0.0,0.9,0.0,0.4]) # 90% if the bus is on time 40% if not. 
arrive_dry.set_all([0,0,0.95,0,1,1,0.05,1])
f = -1 # need something other than nan for values which cannot be true. Doesn't matter what this variable is.
arrive_late.set_all([0.95,0.1,f,0.95,0.05,0.9,f,0.05])
utility_factor = factors.Factor(["dry","late"],[2,2])
utility_factor.set_all([-2,-7,0,-5])
utility_relevant_nodes = [arrive_dry,arrive_late]
non_utility_relevent_factors = [rain_factor,bus_factor,catch_bus]

In [90]:
def run_decision_network(decision_node,utility_nodes,utility_factor,other_nodes,evidence_vars=[],evidence_vals=[]):
    max_utility = -np.inf
    best_decision = -1
    print("decision",decision_node.names)
    print("evidence",dict(zip(evidence_vars,evidence_vals)))
    for option in decision_node.indexes:
        decision_node.array = np.zeros(decision_node.array.shape)
        decision_node.set(option,1)
        var_names_to_marginalize = [node.names[0] for node in [decision_node]+other_nodes]
        all_factors = [decision_node]+utility_nodes+other_nodes
        probs = sum_product_variable_elimination(all_factors,evidence_vars,evidence_vals,var_names_to_marginalize)
        total_utility = np.sum(factors.product(probs,utility_factor).array)
        print("choice",option,"UTILITY:",total_utility.round(5))
        if(total_utility>max_utility):
            max_utility = total_utility
            best_decision = option
    return best_decision,max_utility

In [94]:
best_decision,max_utility = run_decision_network(decision,
                             utility_relevant_nodes,
                             utility_factor,
                             non_utility_relevent_factors,
                             evidence_vars=["it will rain"],
                             evidence_vals=[0])
print("best decision",best_decision)

decision ['try to catch the bus']
evidence {'it will rain': 0}
choice [0] UTILITY: -0.25
choice [1] UTILITY: -0.99375
best decision [0]


In [95]:
best_decision,max_utility = run_decision_network(decision,
                             utility_relevant_nodes,
                             utility_factor,
                             non_utility_relevent_factors,
                             evidence_vars=["it will rain"],
                             evidence_vals=[1])
print("best decision",best_decision)

decision ['try to catch the bus']
evidence {'it will rain': 1}
choice [0] UTILITY: -2.15
choice [1] UTILITY: -1.32625
best decision [1]


In [96]:
best_decision,max_utility = run_decision_network(decision,
                             utility_relevant_nodes,
                             utility_factor,
                             non_utility_relevent_factors)
print("best decision",best_decision)

decision ['try to catch the bus']
evidence {}
choice [0] UTILITY: -0.82
choice [1] UTILITY: -1.0935
best decision [0]
